# VE DG and DeepMoD testing loop

In [1]:
import os
import sys
from datetime import datetime
import numpy as np
import torch

sys.path.append('../src')
import deepymod_torch.VE_datagen as VE_datagen
import deepymod_torch.VE_params as VE_params
from deepymod_torch.DeepMod import DeepMoD
from deepymod_torch.library_function import mech_library

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Here, we set many parameters that define our problem. We define:
- The model and model type (which is determined by input_type)
- The shape, frequency and magnitude of the sinc input manipulation
- The sampling rate from the data that will be generated

I also here define any other parameters that need to be defined (for saving or the normal flow) but are not being used or tested for this particular use of the notebook.

In [2]:
investigated_param = 'Varying tau for real imitation'

input_type = 'Strain'
E = [5e-4, 5e-4, 5e-4]
eta = [2.2e-4, 1] # For this test, we vary the second value of eta, so I assign it a nonsense value of 1 here.

func_desc = 'Sinc'
omega = 2*np.pi
Amp = 7
input_expr = lambda t: Amp*np.sin(omega*t)/(omega*t)
d_input_expr = lambda t: (Amp/t)*(np.cos(omega*t) - np.sin(omega*t)/(omega*t))
input_torch_lambda = lambda t: Amp*torch.sin(omega*t)/(omega*t)

number_of_samples = 1000
noise_level = 0

To generate data, we need to choose where and when to evaluate the target data for a given manipulation. Below, we choose those time points.

In [3]:
time_array = np.linspace(10**-10, 10*np.pi/omega, 5000).reshape(-1, 1)

We the scale factor for the x axis data (time) as this is determined by our manipulation description only, not the synthetic response. We can also apply the scaling to the time_array provided we keep the original array also for calculating the synthetic response.

scaled_omega is the effective omega of manipulation after scaling.

In [ ]:
scaled_omega = 1.2
t_sf = omega/scaled_omega
scaled_time_array = time_array*t_sf

Next we configure DeepMoD. We configure:
- The initial L1 regularisation penalty
- The learning rate
- The number of epochs at each stage of training
- The size and shape of the network
- The library function for calculating potential terms relevant to VE problems.

We do not yet put the final lambda function into library_config. This must be added in the loop as it relies on knowing the time scale factor.

In [5]:
optim_config = {'lambda': 10**-6, 'lr_coeffs': 0.002, 'max_iterations': 100001, 'mse_only_iterations': 20001, 'final_run_iterations': 10001}
network_config = {'input_dim': 1, 'hidden_dim': 30, 'layers': 4, 'output_dim': 1}
library_config = {'type': mech_library, 'diff_order': 3, 'coeff_sign': 'positive', 'input_type': input_type}#, 'input_expr': input_torch_lambda}

Information for saving that doesn't change each loop....

In [6]:
parent_folder = '../data/Results'
first_subfolder = investigated_param.replace('.', '-')

Here we define the values we wish to test.

In this use of the notebook it is the value of one of the viscosities used to define our model. We make a geometric progression of values to test a range of magnitudes.

In [7]:
eta_2_values = np.geomspace(2.2e-4, 1e-1, num=11)
eta_2_values

array([0.00022   , 0.00040568, 0.00074806, 0.00137942, 0.00254363,
       0.00469042, 0.00864906, 0.01594876, 0.02940929, 0.05423033,
       0.1       ])

Next we run the loop. In each iteration of the loop:

- Data will be synthesised from the model and manipulation description.
- The data will be prepared for DeepMoD injection
- DeepMoD will try its best
- The results will be organised and saved in a named folder.
- The progress will be available in Tensorboard files also which will need to be manually dragged across after the loop is done (or during!)

The total number of tests are now (relevant when looping 2 sets of values):

In [8]:
# len(tau_2_values)*len(noise_level_values)*number_of_seeds

Main loop!

In [9]:
for param_val in eta_2_values:
    
    print('Starting loop with parameter value:', param_val)
    
    # Update investigated parameter
    eta[-1] = param_val
    
    # reset randomised initialisation
    np.random.seed(2)
    torch.manual_seed(3)

    # DATA GENERATION
    # Generate data using updated model
    strain_array, stress_array = VE_datagen.calculate_strain_stress(input_type, time_array, input_expr, E, eta, D_input_lambda=d_input_expr)
    
    # Scale data (y only)
    strain_sf = 1/np.max(abs(strain_array))
    stress_sf = 1/np.max(abs(stress_array))
    if input_type == 'Strain':
        scaled_input_torch_lambda = lambda t: strain_sf*input_torch_lambda(t/t_sf)
        scaled_target_array = stress_array*stress_sf
    elif input_type == 'Stress':
        scaled_input_torch_lambda = lambda t: stress_sf*input_torch_lambda(t/t_sf)
        scaled_target_array = strain_array*strain_sf
    
    # Add noise to value at each time point
#     noisy_strain_array = strain_array + noise_level * np.std(strain_array) * np.random.standard_normal(strain_array.shape)

    # randomly sample
    reordered_row_indices = np.random.permutation(time_array.size)
    reduced_time_array = scaled_time_array[reordered_row_indices, :][:number_of_samples]
    reduced_target_array = scaled_target_array[reordered_row_indices, :][:number_of_samples]


    # DEEPMOD PREPARATION
    # convert to tensors
    time_tensor = torch.tensor(reduced_time_array, dtype=torch.float32, requires_grad=True)
    target_tensor = torch.tensor(reduced_target_array, dtype=torch.float32)
    
    # load redefined torch expression for input based on change in omega into config
    library_config['input_expr'] = scaled_input_torch_lambda

    
    # DEEPMOD
    # record start time for later transfer of tensorboard files to correct folders
    # start time is always in GMT, regardless of system clock it seems....
    now = datetime.now()
    dt_string = now.strftime('%d/%m/%Y %H:%M:%S')

    # run DeepMoD
    print('Running DeepMoD')
    sparse_coeff_vector_list_list, scaled_coeff_vector_list_list, sparsity_mask_list_list, network = DeepMoD(time_tensor, target_tensor, network_config, library_config, optim_config)
    print('Saving results')


    # ORGANISING RESULTS
    # Calculate unscaled expected coeffs.
    if input_type == 'Stress':
        unscaled_expected_coeffs = VE_params.coeffs_from_model_params_kelvin(E, eta)
    elif input_type == 'Strain':
        unscaled_expected_coeffs = VE_params.coeffs_from_model_params_maxwell(E, eta)

    # Scale true coeffs to ones we expect to be found after scaling.
    # Convert list of expected coeffs to array for saving
    scaled_expected_coeffs = VE_params.scaled_coeffs_from_true(unscaled_expected_coeffs, t_sf, strain_sf, stress_sf)
    target_coeffs_array = np.array(scaled_expected_coeffs).reshape(-1,1)
    
    # recalculate prediction from trained network and convert to array for saving
    prediction_array = np.array(network(time_tensor).detach().cpu())
    
    # convert pre-thresholding coeffs data to arrays for saving
    pre_thresh_coeffs_array = np.array(sparse_coeff_vector_list_list[0][0].detach().cpu())
    pre_thresh_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[0][0].detach().cpu())

    # convert final coeffs data to arrays for saving
    final_coeffs_array = np.array(sparse_coeff_vector_list_list[-1][0].detach().cpu())
    final_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[-1][0].detach().cpu())
    sparsity_mask_array = np.array(sparsity_mask_list_list[-1][0].cpu()).reshape(-1,1)

    # group like data vectors together for saving
    dg_series_data = np.concatenate((time_array, strain_array, stress_array), axis=1)
    NN_series_data = np.concatenate((reduced_time_array, reduced_target_array, prediction_array), axis=1)
    pre_thresh_coeffs_data = np.concatenate((pre_thresh_coeffs_array, pre_thresh_scaled_coeffs_array), axis=1)
    final_coeffs_data = np.concatenate((final_coeffs_array, final_scaled_coeffs_array, sparsity_mask_array), axis=1)
    
    # remove library from dictionary as we don't want to save that
    input_theta = library_config.pop('input_theta')
    
    # Gather miscellaneous information into lists for saving
    dg_info_list = ['E: '+str(E), 'eta: '+str(eta), 'Input: '+input_type, 'Desc: '+func_desc, 'omega: '+str(omega), 'Amp: '+str(Amp)]
    treatment_info_list = ['noise_factor: '+str(noise_level), 'time_sf: '+str(t_sf), 'strain_sf: '+str(strain_sf), 'stress_sf: '+str(stress_sf)]
    config_dict_list = ['optim: '+str(optim_config), 'network: '+str(network_config), 'library: '+str(library_config)]
    misc_list = ['date_stamp: '+dt_string]


    # SAVING RESULTS
    # collect parameters to name folder for saving
    second_subfolder = 'param_' + str(param_val).replace('.', '-')
#     third_subfolder = 'noise_' + str(noise_level).replace('.', '-')
#     fourth_subfolder = 'seed_' + str(seed_value)
    foldername = parent_folder + '/' + first_subfolder + '/' + second_subfolder# + '/' + third_subfolder + '/' + fourth_subfolder

    # make folder
    if not os.path.isdir(foldername):
        os.makedirs(foldername)

    # save all array data
    np.savetxt(foldername+'/DG_series_data.csv', dg_series_data, delimiter=',', header='Time, Strain, Stress')
    np.savetxt(foldername+'/NN_series_data.csv', NN_series_data, delimiter=',', header='Time, Target, Prediction')
    np.savetxt(foldername+'/expected_coeffs.csv', target_coeffs_array, delimiter=',', header='Expected_coeffs')
    np.savetxt(foldername+'/pre_thresh_coeffs_data.csv', pre_thresh_coeffs_data, delimiter=',', header='Trained_Coeffs, Scaled_Trained_Coeffs')
    np.savetxt(foldername+'/final_coeffs_data.csv', final_coeffs_data, delimiter=',', header='Trained_Coeffs, Scaled_Trained_Coeffs, Sparsity_Mask')
    
    # save all lists data
    with open(foldername+'/DG_info_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in dg_info_list)
        
    with open(foldername+'/treatment_info_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in treatment_info_list)
    
    with open(foldername+'/config_dict_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in config_dict_list)
    
    with open(foldername+'/misc_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in misc_list)

Epoch | Total loss | MSE | PI | L1 
10000 4.1E-06 2.0E-06 2.1E-06 0.0E+00
tensor([[0.2825],
        [0.4274],
        [1.0014]], requires_grad=True)
Time elapsed: 4.0 minutes 7.94896388053894 seconds
Saving results
